<a href="https://colab.research.google.com/github/ogut77/DataScience/blob/main/BoostingClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
from matplotlib import pyplot as plt
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
df = pd.read_csv(DATA_PATH + "telecom_churn.csv")

df["International plan"] = pd.factorize(df["International plan"])[0]
df["Voice mail plan"] = pd.factorize(df["Voice mail plan"])[0]
df["Churn"] = df["Churn"].astype("int")
states = df["State"]
y = df["Churn"]
X=df.drop(["State", "Churn"], axis=1)


In [4]:
X

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,0,0,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2
3329,68,415,0,1,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3
3330,28,510,0,1,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2
3331,184,510,1,1,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2


In [15]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=17
)



DecisionTreeClassifier()

In [29]:
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(random_state=17)
tree.fit(X_train, y_train)
# make predictions for test data
tree_pred = tree.predict(X_test)
accuracy_score(y_test, tree_pred) 

0.92

In [30]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=17 )
forest.fit(X_train, y_train)
# make predictions for test data
forest_pred = forest.predict(X_test)
accuracy_score(y_test, forest_pred) 

0.95

In [33]:
# fit model no training data
from xgboost import XGBClassifier
model = XGBClassifier(random_state=17)
model.fit(X_train, y_train)
# make predictions for test data
xgb_pred = model.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.951

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
from time import time
t0 = time()
gbm_model = GradientBoostingClassifier(random_state=17)
gbm_model.fit(X_train, y_train)
gbm_time = time() - t0
accuracy_score(y_test, gbm_model.predict(X_test))

0.954

In [37]:
import lightgbm as lgb
t0 = time()
lgb_model = lgb.LGBMClassifier(random_state=17)
lgb_model.fit(X_train, y_train)
lgb_time = time() - t0
accuracy_score(y_test, lgb_model.predict(X_test))

0.956

In [ ]:
!pip install catboost


In [47]:
from catboost import CatBoostClassifier
cbc= CatBoostClassifier(random_state=17,verbose=False)

cbc.fit(X_train, y_train)
accuracy_score(y_test, cbc.predict(X_test))


0.955

In [65]:
combined=np.round((cbc.predict_proba(X_test)[:,1]+lgb_model.predict_proba(X_test)[:,1]+gbm_model.predict_proba(X_test)[:,1])/3)
accuracy_score(y_test, combined)

0.957